## Repeat Purchase Project
#### This will use Dunnhumby data to predict the repurchase chance of existing customers. 
Machine learning will be used. Much of the work here will be creating a data set and a model to fit the data set. RFM will be important here as these will be indicators/features we look out for.

Link to XGBoost Tutorial on Towards Data Science:
https://towardsdatascience.com/getting-started-with-xgboost-in-scikit-learn-f69f5f470a97

In [176]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np

### Now we load transaction, product, consumer csv's

In [177]:
transaction_df = pd.read_csv('../Resources/dunnhumby/transaction_data.csv')
product_df = pd.read_csv('../Resources/dunnhumby/product.csv')
hh_df = pd.read_csv('../Resources/dunnhumby/hh_demographic.csv')

In [178]:
transaction_df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [179]:
product_df.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [180]:
hh_df.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


#### We could do some quick summaries using buckets just to see what the data as a whole looks like. Let's take a look at the consumer data, first.

In [181]:
hh_df.groupby(['AGE_DESC']).count()

,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
AGE_DESC,,,,,,,
19-24,46,46,46,46,46,46,46
25-34,142,142,142,142,142,142,142
35-44,194,194,194,194,194,194,194
45-54,288,288,288,288,288,288,288
55-64,59,59,59,59,59,59,59
65+,72,72,72,72,72,72,72


#### We can tell from the aboce data that the big spenders/the largest group of consumers is 45-54, and 45-54 as a runner-up.

In [182]:
hh_df.groupby(['INCOME_DESC']).count()

,AGE_DESC,MARITAL_STATUS_CODE,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
INCOME_DESC,,,,,,,
100-124K,34,34,34,34,34,34,34
125-149K,38,38,38,38,38,38,38
15-24K,74,74,74,74,74,74,74
150-174K,30,30,30,30,30,30,30
175-199K,11,11,11,11,11,11,11
200-249K,5,5,5,5,5,5,5
25-34K,77,77,77,77,77,77,77
250K+,11,11,11,11,11,11,11
35-49K,172,172,172,172,172,172,172


In [183]:
hh_df.groupby(['INCOME_DESC']).count()['household_key'].sum()

801

#### Majority of our consumers here are under 50k a year. We could compare that income to number of people per house, kids, marital status, etc, but we can leave those as features for our model that we will end up building.

In order to get into modeling, we have to process the data in such a way that we organize the households into one time purchasers and multiple time purchasers. We can do this through the transaction dataframe, where we count, for each unique household key, how many times that household occurs. gather the data for those households, and assemble a numpy array that can be passed into an XGBClassifier. This classifier will be provided the data of customers who have purchased multiple times and what their behavior is, and then will be used to predict on the households who have only purchased once to see who is most likely. The XGBClassifier will return values ranging between 0 and 1, which will reflect a probability or confidence of repurchase. We could then have an algorithm process the predictions and report back who exceeded a certain threshold based on what amount of confidence we ask that algorithm specifically to look for.

In [184]:
transaction_df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [185]:
'''
the way the data was collected, there are multiple times households may show up that are really part of the same
single transaction, so to avoid that, and since household key wont change if we take the average for each 
basket id which is unique to that transaction, so we can group by basket id to consolidate a transaction, and then
once again count for a household key so that we can get a true count of the number of transactions per household.
this is all to determine if households have come back to purchase. 
We also will want to separate purchases by store. If we consolidate all purchases and make an assumption that these 
are all with one brand, we sort of dirty the waters and ignore the fact that these customers are making return 
purchases to specific brands which in and of itself is a decision the consumer is making.'''
transaction_df.groupby(['BASKET_ID']).mean()

,household_key,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
BASKET_ID,,,,,,,,,,,
26984851472,2375.0,1.0,2.463398e+06,1.000000,1.182000,364.0,-0.258000,1631.0,1.0,0.0,0.0
26984851516,2375.0,1.0,3.328273e+06,1.166667,2.071667,364.0,-0.543333,1642.0,1.0,0.0,0.0
26984896261,1364.0,1.0,9.160190e+05,1.000000,2.274000,31742.0,-0.436000,1520.0,1.0,0.0,0.0
26984905972,1130.0,1.0,9.686606e+05,1.800000,0.510000,31642.0,-0.416000,1340.0,1.0,0.0,0.0
26984945254,1173.0,1.0,9.599073e+05,1.333333,1.176667,412.0,0.000000,2042.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
42302712006,2262.0,711.0,4.290221e+06,1.000000,2.445000,446.0,-0.447500,1652.0,102.0,0.0,0.0
42302712189,1369.0,711.0,2.661998e+06,1.200000,6.728000,446.0,-0.220000,1730.0,102.0,0.0,0.0
42302712298,2225.0,711.0,3.756731e+06,1.000000,3.700000,446.0,-0.035455,1754.0,102.0,0.0,0.0


In [186]:
#number of stores
len(transaction_df.groupby(['BASKET_ID']).mean().groupby(['STORE_ID']).mean())

582

In [187]:
merged_hh = pd.merge(hh_df, transaction_df.groupby(['household_key']).count()['QUANTITY'], on = "household_key")
''' 
so we have matched each household with the number of products they purchased. This should allow some
decent visualization/organization. also our XGBClassifier should be able to take this 'QUANTITY' value into
account and evaluate liklihood of repurchase based on that as well.
'''
#for the next step we are going to swap the KID_CATEGORY_DESC 'None/Unknown' value for 0
merged_hh['KID_CATEGORY_DESC'] = merged_hh['KID_CATEGORY_DESC'].replace({'None/Unknown':0})

#converting the homeowner category to one-hot-encoding
from keras.utils import to_categorical
merged_hh['HOMEOWNER_DESC_TO_INT'] = merged_hh['HOMEOWNER_DESC']
merged_hh['HOMEOWNER_DESC_TO_INT'] = merged_hh['HOMEOWNER_DESC_TO_INT'].replace({'Homeowner':0,
                                                                                'Unknown':1,
                                                                                'Renter':2,
                                                                                'Probable Renter':3,
                                                                                'Probable Owner':4})
HOMEOWNER_NP = merged_hh['HOMEOWNER_DESC_TO_INT'].to_numpy()
HOMEOWNER_TF = to_categorical(HOMEOWNER_NP, num_classes = 5)
merged_hh['HOMEOWNER_BINARY'] = HOMEOWNER_TF[:, 0]
merged_hh['HOMEOWNER_BINARY'] = merged_hh['HOMEOWNER_BINARY'].astype('int32')
merged_hh['UNKNOWN_BINARY'] = HOMEOWNER_TF[:, 1]
merged_hh['UNKNOWN_BINARY'] = merged_hh['UNKNOWN_BINARY'].astype('int32')
merged_hh['RENTER_BINARY'] = HOMEOWNER_TF[:, 2]
merged_hh['RENTER_BINARY'] = merged_hh['RENTER_BINARY'].astype('int32')
merged_hh['PROBABLE_RENTER_BINARY'] = HOMEOWNER_TF[:, 3]
merged_hh['PROBABLE_RENTER_BINARY'] = merged_hh['PROBABLE_RENTER_BINARY'].astype('int32')
merged_hh['PROBABLE_OWNER_BINARY'] = HOMEOWNER_TF[:, 4]
merged_hh['PROBABLE_OWNER_BINARY'] = merged_hh['PROBABLE_OWNER_BINARY'].astype('int32')
merged_hh['HOUSEHOLD_SIZE_DESC'] = merged_hh['HOUSEHOLD_SIZE_DESC'].replace({'5+':5})
merged_hh['KID_CATEGORY_DESC'] = merged_hh['KID_CATEGORY_DESC'].replace({'3+':3})
merged_hh['KID_CATEGORY_DESC'] = merged_hh['KID_CATEGORY_DESC'].astype('int32')
merged_hh['HOUSEHOLD_SIZE_DESC'] = merged_hh['HOUSEHOLD_SIZE_DESC'].astype('int32')
merged_hh['ADULTS'] = merged_hh['HOUSEHOLD_SIZE_DESC'].astype('int32') - merged_hh['KID_CATEGORY_DESC'].astype('int32')
merged_hh = merged_hh.rename(columns = {'KID_CATEGORY_DESC':'KIDS',
                                       'HOUSEHOLD_SIZE_DESC':'HOUSEHOLD_SIZE',
                                       'QUANTITY':'NUMBER_OF_PURCHASES'})
merged_hh.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE,KIDS,household_key,NUMBER_OF_PURCHASES,HOMEOWNER_DESC_TO_INT,HOMEOWNER_BINARY,UNKNOWN_BINARY,RENTER_BINARY,PROBABLE_RENTER_BINARY,PROBABLE_OWNER_BINARY,ADULTS
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,0,1,1727,0,1,0,0,0,0,2
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,0,7,1286,0,1,0,0,0,0,2
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,1979,1,0,1,0,0,0,2
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,2348,0,1,0,0,0,0,2
4,45-54,B,50-74K,Homeowner,Single Female,1,0,16,517,0,1,0,0,0,0,1


In [188]:
'''
we want to now merge this data about households with transactions per store, maybe keep a dictionary of store data
that is separated by store so that we can feed each one into the model and train it further. so we can have a database
of transactions by store, where each transaction has consumer data. i actually still have yet to figure out whether
we feed the model consumers and decide whether or not those consumers should be classified as returning customers OR
whether we feed them transaction data for which case we have the store predict which consumer will be the next
transaction. although speaking outloud it seems it might be best to feed it a consumer data base with number of 
purchases made to each store. so maybe we can still do it by having a database of consumer data with all the purchases
they made to a specific store. i guess in that way we wouldnt have to one-hot encode for every store (that would
be a lot of columns). another thing, should we combine this with product information? we could one-hot that which
be quite a lot of that but could be interesting. or at least when it came to what type of products each transaction
focused on. we have that data.
'''
full_transaction_df = pd.merge(merged_hh, transaction_df, on = 'household_key')
full_transaction_df = full_transaction_df.sort_values(by='WEEK_NO')
stores_dict = {} #going to create a dictionary with one dataframe per store
for ID in full_transaction_df['STORE_ID'].unique():
    stores_dict[ID] = full_transaction_df.loc[(full_transaction_df['STORE_ID'] == ID),:]
    stores_dict[ID]['PURCHASED_FROM_THIS_STORE'] = transaction_df.groupby(['BASKET_ID'].mean().loc[transaction_df['STORE_ID'] == ID,['']]
stores_dict
'''going to drop columns from the transaction lists so that we can get it ready for the ML portion of this project.
we are going to drop:
- STORE_ID: dropping this because we already split up the transaction dataframe into the dictionary above
- MARITAL_STATUS_CODE: honestly i dont know what this is and i want to avoid getting bogged down in something that doesn't
have a whole lot of impact. id say this is taking a risk in order to speed up getting a result.
- HH_COMP_DESC: we already split this up into various categories
- COUPON_MATCH_DISC: this is in the form of a percentage discount that is matched against some outside source. I just
feel like sorting through this is not very worth it based on how much value I can see it providing.
- HOMEOWNER_DESC_TO_INT: i dont know what this is either
'''
for store in stores_dict:
    stores_dict[store].drop(columns = ['STORE_ID','MARITAL_STATUS_CODE','HH_COMP_DESC','HOMEOWNER_DESC_TO_INT','COUPON_MATCH_DISC'], inplace = True)
stores_dict[364]

SyntaxError: invalid syntax (<ipython-input-188-b20601b91501>, line 20)

In [ ]:
stores_dict[364].columns

## *STARTING OVER*
Starting over because I realized I'm doing this in a way that I don't feel is right. I split the transactions with regards to stores which I think is appropriate, but I have way too many variables. I think I started with way too information instead of first brainstorming what I think would actually be important in indicating what will lead to repurchase. On one hand, brainstorming first can lead to some variables being left out that would actually be pretty important such as coupon discount, sale discount total (the actual amount of money saved from the coupon), the week number, etc. But on the other hand, I think brainstorming can help provide direction and can help keep the task manageable. On top of this, it's always possible to engineer more categories. 

##### What To Do Next?
So I think I will first start by removing categories that I think are unnecessary. Then I will split the dataframes first into separate stores, then group by households. This way, we will train the ML algorithm to look at each household's transaction history and determine for that household the confidence of repurchase for that particular store.

##### Which categories are important?
I think the amount and different types of people within the household are important.I could do something where we determine how many purchases each household has made along with some bits of info on the household and use that to determine repurchase confidence, but I think that's less accurate because it doesn't take into account whether the transactions they made were based on whether there was a sale, how often those transactions were, etc. 
- Time stamps I think are unnecessary because I don't think the fact someone purchased groceries at 7 in the morning will mean anything when it's been a week since that person has bought groceries and may need more. 
- income and age I think can be significant because perhaps a certain purchase was a gift and that household may or may not purchase from that store again because that store personally does not appeal to them but may appeal to the person they purchased the gift for, therefore limiting the amount of times that household returns to the store to purchase anything. 
- household owners or renters can be important, might be overshadowed by income range but ownership status may be a personal choice and reflect some hidden personality pattern that would otherwise be hidden if we did not include ownership status
- which products were purchased I think are important because certain products may not need to be purchased as often as others, such as salt versus eggs, etc. The amount of each product purchased I think will also be important. The way the transaction dataframe is set up is that each item purchased is listed versus each cart/full transaction. So each item purchased has a basket ID that contained that particular item. With this in mind, i may actually want to approach this as a transaction-based list, where each transaction has each product listed and the quantity of those products listed as well. this way it will be a more accurate way of representing how often this particular household comes in to purchase goods.
- Sales value and other discount related categories are very significant because a particular household's transactions may consist primarily of discounted purchases and thus may only be a patron if there are coupons and discounts available. 

##### Which Categories Should I drop?
I think any category that gets too granular and/or doesn't actually represent anything relevant to a transaction such as previous number of items purchased, etc.

##### Outstanding Questions and Thoughts
My first concern is how the ML algorithm will absorb/process the number of transactions each household has made. Some households have over 700 transactions, whereas some literally only have 1. So in terms of how to separate these and/or take this into account, I guess I will find out as I approach actually loading the data into the XGBoost algorithm. Maybe I will only train on households with transactions over 500, and then test it on everyone, but then how will I put in 1 transaction from a household into the algorithm and have that data be the same shape as a household with 500 purchases? I could enter in N/A for every transaction that hasn't happened but now I'm wondering if that will add some unintended weight to certain cateories. I could simplify this and have the algorithm only train on household info and train on every single household that purchased for a particular store and feed it how many purchases each household made at that store, and then have it predict whether a household will repurchase based on previous number of transactions at which store and which products were bought and how much, *and that actually would have a much wider application considering that would be a great way to forecast sales based on previous customers, help create estimates for a loyalty program, and other applications to other problems that this whole "10 retail-based data science projects" wants to cover.* Maybe we actually do that instead^. The main issue we face is the fact that we haven't yet decided how to determine if that customer made a repeat purchase. Maybe it's as simple as asking if this household has purchased more than once from this store, and if so it's a repeat customer. But then how do we test it? the tricky part here is the fact that any customer could stop coming to the store at any point and there would be no indication that this customer never comes back. But maybe the concern here is to worry about people who have only made one purchase. Maybe the point here is to leave whether this customer returns after a couple purchases up to them, but once they exceed a threshold that's when you start marketing to them? These are all questions that lead to other parts of retail which is valuable, but I also want to make a point of sticking to repurchase. 

##### Alright, So What The Fuck Are We Doing, Then? Have We Decided?
Based on the conclusion that I reached after writing out my thoughts, I've decided that I'm going to train the algorithm on each household that patronized each store. So, I want to go over what the data will look like. In no particular order, though I want to put emphasis on the categories that come first here, the columns will include:

- Household size (This includes all the sex and kid # categories)
- Household age
- Household income
- Ownership status
- Number of purchases at each store
- Number of each product purchased (products that have zero purchases across the board should be eliminated from that store's dataframe considering that item might not exist at that store)
- Total amount saved via coupons (this will come in three categories, the first being just flat-out how much did this person save in terms of raw dollars. the second category will be the amount this person saved with respect to how much they spent. A larger saved to paid ratio will indicate a larger concern to save. And the third category will be the ratio of transactions that used coupons to transactions that did not. A larger ratio will indicate the same patterns as the second category)
- The only time-based data point for each household, the number of days (or weeks, I haven't looked at the data that closely, whichever will be more accurate) since their last purchase. 

#### Plan Of Attack
The following cells will be gathering the data in the exact order that you see in the above list. I will block each chunk off, and try to explain what I'm doing each step of the way. We will begin with the stores_dict and organize it so we obtain the information we want. we will shave off the categories and columns we don't need, and check for any null values.

In [189]:
merged_hh.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE,KIDS,household_key,NUMBER_OF_PURCHASES,HOMEOWNER_DESC_TO_INT,HOMEOWNER_BINARY,UNKNOWN_BINARY,RENTER_BINARY,PROBABLE_RENTER_BINARY,PROBABLE_OWNER_BINARY,ADULTS
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,0,1,1727,0,1,0,0,0,0,2
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,0,7,1286,0,1,0,0,0,0,2
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,1979,1,0,1,0,0,0,2
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,2348,0,1,0,0,0,0,2
4,45-54,B,50-74K,Homeowner,Single Female,1,0,16,517,0,1,0,0,0,0,1


In [190]:
stores_dict[364]

,INCOME,household_key,AGE,QUANTITY_PRODUCTS_PURCHASED,HOUSEHOLD_SIZE,KIDS,HOMEOWNER_BINARY,UNKNOWN_BINARY,RENTER_BINARY,PROBABLE_RENTER_BINARY,PROBABLE_OWNER_BINARY,ADULTS
0,15.0,67.0,49.5,2.0,5,3,1,0,0,0,0,2
1,87.0,97.0,49.5,110.0,1,0,0,1,0,0,0,1
2,42.0,354.0,29.5,12.0,5,3,1,0,0,0,0,2
3,29.5,432.0,21.5,39.0,1,0,0,1,0,0,0,1
4,19.5,442.0,29.5,2.0,2,1,0,1,0,0,0,1
5,42.0,586.0,21.5,1.0,3,1,0,0,1,0,0,2
6,62.0,764.0,39.5,1.0,5,3,1,0,0,0,0,2
7,42.0,979.0,39.5,42.0,2,1,1,0,0,0,0,1
8,29.5,1066.0,39.5,231.0,2,0,0,1,0,0,0,2
9,62.0,1179.0,39.5,2.0,2,0,1,0,0,0,0,2


##### We will use stores_dict data to determine:
- number of products each household has purchased. 
- amount saved from each product bought and accumulate that into one large sum for each household

In [191]:
merged_hh.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE,KIDS,household_key,NUMBER_OF_PURCHASES,HOMEOWNER_DESC_TO_INT,HOMEOWNER_BINARY,UNKNOWN_BINARY,RENTER_BINARY,PROBABLE_RENTER_BINARY,PROBABLE_OWNER_BINARY,ADULTS
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,0,1,1727,0,1,0,0,0,0,2
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,0,7,1286,0,1,0,0,0,0,2
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,1979,1,0,1,0,0,0,2
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,2348,0,1,0,0,0,0,2
4,45-54,B,50-74K,Homeowner,Single Female,1,0,16,517,0,1,0,0,0,0,1


In [192]:
stores_dict = {} #going to create a dictionary with one dataframe per store
#this loop here finds all unique stores, and finds out which households purchased from each store, and how many items
#that household purchased
for ID in full_transaction_df['STORE_ID'].unique():
    basket_df = transaction_df.groupby(['BASKET_ID']).mean()
    #the line below both finds all households that purchased from this specific store, but also how many items
    new_df = basket_df.loc[basket_df['STORE_ID'] == ID,:].groupby(['household_key']).count()
    #we can use new_df as a sort of list of stores for the specific store it is storing for in stores_dict,
    #as well as how many items each household purchased at this store. We can then merge on the left so that
    #only the households in this list get added to this dataframe, so it's much easier to keep track of.
    new_df = new_df[['QUANTITY']]
    new_df.rename(columns = {'QUANTITY':
                            'QUANTITY_PRODUCTS_PURCHASED'}, inplace = True)
    stores_dict[ID] = pd.merge(new_df, merged_hh, on = 'household_key', how = 'right')
    stores_dict[ID].drop(columns = ['MARITAL_STATUS_CODE',
                                    'HOMEOWNER_DESC_TO_INT',
                                    'NUMBER_OF_PURCHASES',
                                    'HH_COMP_DESC',
                                    'HOMEOWNER_DESC'], inplace = True)
    stores_dict[ID].dropna(how='any', inplace = True) #any N/A values were households that did not purchase anything
                                                        #from this specific store
stores_dict[364]

,household_key,QUANTITY_PRODUCTS_PURCHASED,AGE_DESC,INCOME_DESC,HOUSEHOLD_SIZE,KIDS,HOMEOWNER_BINARY,UNKNOWN_BINARY,RENTER_BINARY,PROBABLE_RENTER_BINARY,PROBABLE_OWNER_BINARY,ADULTS
0,67.0,2.0,45-54,Under 15K,5,3,1,0,0,0,0,2
1,97.0,110.0,45-54,75-99K,1,0,0,1,0,0,0,1
2,354.0,12.0,25-34,35-49K,5,3,1,0,0,0,0,2
3,432.0,39.0,19-24,25-34K,1,0,0,1,0,0,0,1
4,442.0,2.0,25-34,15-24K,2,1,0,1,0,0,0,1
5,586.0,1.0,19-24,35-49K,3,1,0,0,1,0,0,2
6,764.0,1.0,35-44,50-74K,5,3,1,0,0,0,0,2
7,979.0,42.0,35-44,35-49K,2,1,1,0,0,0,0,1
8,1066.0,231.0,35-44,25-34K,2,0,0,1,0,0,0,2
9,1179.0,2.0,35-44,50-74K,2,0,1,0,0,0,0,2


Next, I'm looking at the AGE_DESC and INCOME_DESC categories. The algorithm won't make approximations, etc, and so I'm thinking in order to make the number matter, we take the average of the two numbers that are used as the boundaries of the range, for both of them.

In [193]:
stores_dict[432]['AGE_DESC'].unique()

array(['35-44', '25-34', '65+', '45-54', '55-64', '19-24'], dtype=object)

In [194]:
def get_age(ID, stores_dict):
    age_df = stores_dict[ID][['AGE_DESC','household_key']]
    age_replace_dict = {}
    for age_range in stores_dict[ID]['AGE_DESC'].unique():
        if age_range == '65+':
            avg = 70 #doesn't matter what the age is
            age_replace_dict[age_range] = avg
            continue #i think this continue statement can replace an else statement
        avg_list = [int(x) for x in age_range.split('-')]
        avg = np.average(avg_list)
        age_replace_dict[age_range] = avg

    age_df.replace(to_replace = age_replace_dict, inplace = True)
    age_df.rename(columns = {'AGE_DESC':'AGE'}, inplace = True)
    return age_df

In [195]:
#this cell will be for extracting the age
for store in stores_dict:
    age_df = get_age(store, stores_dict)
    stores_dict[store] = pd.merge(age_df, stores_dict[store], on = 'household_key', how='right')
    stores_dict[store].drop(columns = ['AGE_DESC'], inplace = True)

In [196]:
#it seems here that we have succesfully extracted age. Now for INCOME_DESC
stores_dict[364].head()

,AGE,household_key,QUANTITY_PRODUCTS_PURCHASED,INCOME_DESC,HOUSEHOLD_SIZE,KIDS,HOMEOWNER_BINARY,UNKNOWN_BINARY,RENTER_BINARY,PROBABLE_RENTER_BINARY,PROBABLE_OWNER_BINARY,ADULTS
0,49.5,67.0,2.0,Under 15K,5,3,1,0,0,0,0,2
1,49.5,97.0,110.0,75-99K,1,0,0,1,0,0,0,1
2,29.5,354.0,12.0,35-49K,5,3,1,0,0,0,0,2
3,21.5,432.0,39.0,25-34K,1,0,0,1,0,0,0,1
4,29.5,442.0,2.0,15-24K,2,1,0,1,0,0,0,1


In [197]:
stores_dict[364]['INCOME_DESC'].unique() #max is '250K+' and min is 'Under 15K'

array(['Under 15K', '75-99K', '35-49K', '25-34K', '15-24K', '50-74K'],
      dtype=object)

In [198]:
#same idea as with age, we can copy and paste the get_age function and adjust it for the income
def get_income(ID, stores_dict):
    income_df = stores_dict[ID][['INCOME_DESC','household_key']]
    income_replace_dict = {}
    for income_range in stores_dict[ID]['INCOME_DESC'].unique():
        if income_range == 'Under 15K':
            avg = 15 #doesn't matter what the age is
            income_replace_dict[income_range] = avg
            continue #i think this continue statement can replace an else statement
        elif income_range == '250K+':
            avg = 250
            income_replace_dict[income_range] = avg
            continue
        avg_list = income_range.split('-')
        avg_list[0] = int(avg_list[0]) #the first element doesn't have 'k' attached
        avg_list[1] = int(avg_list[1].split('K')[0])
        avg = np.average(avg_list)
        income_replace_dict[income_range] = avg

    income_df.replace(to_replace = income_replace_dict, inplace = True)
    income_df.rename(columns = {'INCOME_DESC':'INCOME'}, inplace = True)
    return income_df

In [199]:
#this cell will be for extracting the age
for store in stores_dict:
    income_df = get_income(store, stores_dict)
    stores_dict[store] = pd.merge(income_df, stores_dict[store], on = 'household_key', how='right')
    stores_dict[store].drop(columns = ['INCOME_DESC'], inplace = True)

In [200]:
#it seems here that we have succesfully extracted income
stores_dict[364].head()

,INCOME,household_key,AGE,QUANTITY_PRODUCTS_PURCHASED,HOUSEHOLD_SIZE,KIDS,HOMEOWNER_BINARY,UNKNOWN_BINARY,RENTER_BINARY,PROBABLE_RENTER_BINARY,PROBABLE_OWNER_BINARY,ADULTS
0,15.0,67.0,49.5,2.0,5,3,1,0,0,0,0,2
1,87.0,97.0,49.5,110.0,1,0,0,1,0,0,0,1
2,42.0,354.0,29.5,12.0,5,3,1,0,0,0,0,2
3,29.5,432.0,21.5,39.0,1,0,0,1,0,0,0,1
4,19.5,442.0,29.5,2.0,2,1,0,1,0,0,0,1


##### Now that we have both income and age, we have rounded out the household info that we wanted. Now, onto the amount of each item each household purchased from each store. 
We will approach this basically through tallying through each individual product id and summing the amount of times each household shows up within that filtered data frame, and then merge that data frame back into the main dictionary for that product ID, for each store. 
Before we start that, maybe we can see which store sells which products, and only merge with stores that sell the product that we want to merge.